In [1]:
import pyodbc
import pandas as pd
import numpy as np
from pprint import pprint
import datetime
from configparser import ConfigParser


In [2]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [3]:
# Changed Code for audit integration
conn.execute("""insert into FINCORE_DB.Fincore_Analytics.AUDIT_INTGN (JOB_RUN_ID,JOB_NAME,JOB_START_DATE,JOB_STATUS,TARGET_TBL,SOURCE_TBL,SOURCE_MAX_DATE)values
(COALESCE((select (max(JOB_RUN_ID)+1) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN),1), 'RISK_PERSON',GETDATE(),'ERROR','RISK_PERSON','PERSON_MASTER_CLUSTER_FINGATE_REPORT',(SELECT MAX(T.MAX_DATE) FROM (SELECT MAX(CREATE_DATE) AS MAX_DATE FROM FINCORE_DB.FINCORE.PERSON_MASTER_CLUSTER_FINGATE_REPORT UNION SELECT MAX(UPDATE_DATE) AS MAX_DATE FROM FINCORE_DB.FINCORE.PERSON_MASTER_CLUSTER_FINGATE_REPORT) T))""")

job_id = pd.read_sql_query("select max(JOB_RUN_ID) as job_run_id from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN",conn)
job_id = job_id.iloc[0,0]
rules_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES]", conn)
conn.commit()
conn.close()


In [19]:
%store job_id

Stored 'job_id' (int64)


In [20]:
job_id

71

In [5]:
#Queries that populate risk_person_summary_stg

master_data_pop_per_q1 = '''MERGE INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG 
WITH (HOLDLOCK) AS TARGET USING   (

SELECT DISTINCT PMCFR.PERSON_MASTER_ID
FROM FINCORE.PERSON_MASTER_CLUSTER_FINGATE_REPORT PMCFR WITH (NOLOCK)
INNER JOIN FINCORE.PERSON_CLUSTER PC  WITH (NOLOCK)
ON PMCFR.PERSON_CLUSTER_ID = PC.PERSON_CLUSTER_ID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_PERSONDETAIL PD WITH (NOLOCK)
ON PC.FINGATE_PERSON_ID = PD.ID_ AND PD.REPORTID = PMCFR.REPORT_ID
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_INCREMENTAL k  WITH (NOLOCK) 
ON K.PERSON_MASTER_ID = PMCFR.PERSON_MASTER_ID
WHERE IS_ACTIVE_FLAG=1 

 )   
    AS SOURCE ON   
    TARGET.PERSON_MASTER_ID = SOURCE.PERSON_MASTER_ID     
    WHEN MATCHED THEN       
    UPDATE SET TARGET.UPDATE_DATE = GETDATE() 
    WHEN NOT MATCHED BY TARGET THEN      
    INSERT (PERSON_MASTER_ID,CREATE_DATE)  
    VALUES (SOURCE.PERSON_MASTER_ID,GETDATE());   '''




per_pincode_update = '''
UPDATE [fincore_db].[Fincore_Analytics].[RISK_person_SUMMARY_STG]
SET PINCODE = B.pin
from 
(select pan as P, pin
from
(
select  pan, try_cast(try_cast(primaryPinCode as int) as decimal(6,0)) as pin,  
rank() over(partition by pan order by legacy_batch_id desc) as orank 
from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_PersonDetail]
where pan is not null and primaryPinCode is not null
) T
where orank= 1 

)
B
where pan = B.P'''



per_occ_update = '''UPDATE   [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG] 
   SET [OCCUPATION_BUSINESS] = LOB

from 
(
SELECT PERSON_MASTER_ID,LOB FROM (Select PERSON_MASTER_ID, 
RANK () OVER (PARTITION BY PERSON_MASTER_ID ORDER BY rcd_create_date DESC ) AS RNK, LOB,
RCD_CREATE_DATE 
FROM
(
--CODE FOR FINCORE 2.0
SELECT A.persON_MASTER_ID,
COALESCE( (
SELECT TITLE FROM [DATAHUB_DB].[FIUMetaHub].finnet_OccupationType with (nolock) 
WHERE ID_ = D.occupationId),'Others') as LOB
,d.rcd_create_date 
FROM
fincore_db.Fincore_Analytics.RISK_PERSON_INCREMENTAL A with (nolock)
INNER JOIN fincore_db.Fincore.person_Master_Cluster_Fingate_Report B with (nolock) ON A.PERSON_MASTER_ID = B.PERSON_MASTER_ID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.fingate_Persondetail D with (nolock) ON B.Batch_Id = D.batchId AND B.Report_Id = D.reportId
WHERE D.migrated_flag is null  


union
--CODE FOR HDM DATA
sELECT k.PERSON_MASTER_ID,LOB,rcd_create_date FROM 
FINCORE_DB.Fincore_Analytics.RISK_PERSON_INCREMENTAL k with (nolock)
inner join FINCORE_DB.Fincore.Person_Master PM with (nolock) on k.PERSON_MASTER_ID = PM.Person_Master_Id
INNER JOIN (
select pan ,  upper(LEGACY_OCCUPATION) as LOB,rcd_create_date
from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_PersonDetail]  with (nolock)
where LEGACY_OCCUPATION is not null  AND migrated_flag = 'y'
group by pan ,  upper(LEGACY_OCCUPATION),rcd_create_date

)A 
ON PM.PAN = A.PAN
 

) Z ) Y 
WHERE Y.RNK = 1
) B
where [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG].person_master_id = B.Person_master_id'''


master_data_pop_per_q2 = '''update Fincore_Analytics.[RISK_PERSON_SUMMARY_STG]

SET PAN = b.pan , 
DIN = b.din ,
full_name = b.full_name,
Father_Name = b.Father_Name,
Date_Of_Birth = b.Date_Of_Birth,
Ckyc_Number = b.Ckyc_Number,
gender = b.gender
from 
(
select pm.Person_Master_Id as entity_id , 
full_name,
Father_Name , 
Date_Of_Birth ,
gender , 
pan , 
din , 
Ckyc_Number
from Fincore.Person_Master pm with(nolock)
inner join fincore_db.fincore_analytics.risk_person_incremental k with(nolock)
on pm.person_master_id = k.person_master_id
) b
where person_master_id = b.entity_id'''



In [6]:
master_data_pop_per_income = '''update [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG]
set cbdt_income_range = b.income_range
from (
select pm.Person_Master_Id as entity_id,
case when itr.Gross_Total_Income_Range = 0 then null
when itr.Gross_Total_Income_Range = 1 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 2 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 3 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 4 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 5 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 6 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 7 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 8 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 9 then 10000000 end as Income_range 
from FINCORE_DB.FINCORE.Person_Master pm with(nolock)
inner join fincore_db.fincore_analytics.risk_person_incremental k with(nolock)
on pm.person_master_id = k.person_master_id
inner join EXTL_DB.CBDT.CBDT_Itr_Data itr with(nolock)
on itr.Pan =pm.pan 
inner join   EXTL_DB.CBDT.CBDT_Income_Range   cbdtir  with(nolock)
on itr.Gross_Total_Income_Range=cbdtir.Income_Range 
group by pm.Person_Master_Id,cbdtir.Max_Value_Lakh,cbdtir.Min_Value_Lakh,Gross_Total_Income_Range
)b

where person_master_id = b.entity_id'''





per_trx_update_credit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set TRAN_AMNT_CREDIT = round(average,2)
from (


select w.Person_Master_Id as Person_Master_Id,sum(w.monthlyaverage)as average
from 
(select PMAM.Person_Master_Id ,
 
 
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM
inner join fincore_db.Fincore.account_master AM on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select accountNumber ,
avg(amount) as average,
RANK() OVER (PARTITION BY accountNumber ORDER BY  transactionDate desc ) Rank
--sum(amount)/Datediff(Month,transactionDate,getdate())  as monthlyaverage
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_Tctwo 
where
accountNumber is not null and accountNumber !=''
and depositeWithdrawl ='deposit'
group by accountNumber,transactionDate)t on t.accountNumber=Am.Account_Number 
group by PMAM.Person_Master_Id
 
 
 
union
 
 
select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM
inner join fincore_db.Fincore.account_master AM on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select accountNumber ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY accountNumber ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TcOne 
where 
accountNumber is not null and accountNumber !=''
and depositWithdrawal ='deposit'
group by accountNumber,transactionDate)t on t.accountNumber=Am.Account_Number 
group by PMAM.Person_Master_Id
 
 

union
 
 

select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM
inner join fincore_db.Fincore.account_master AM on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select beneficiaryAcNo ,
AVG(Amount) as average,
RANK() OVER (PARTITION BY beneficiaryAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsOne 
where  
beneficiaryAcNo is not null and beneficiaryAcNo !=''
and transactionType ='credit'
group by beneficiaryAcNo ,transactionDate)t
on t.beneficiaryAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.Person_Master_Id
 
 

union
 
 

select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM
inner join fincore_db.Fincore.account_master AM on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select beneficiaryAcNo ,
AVG (amount) as average,
RANK() OVER (PARTITION BY beneficiaryAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsTwo 
where 
beneficiaryAcNo is not null and beneficiaryAcNo !=''
and transactionType ='credit'
group by beneficiaryAcNo ,transactionDate)t
on t.beneficiaryAcNo=Am.Account_Number 
where Rank  = 1
group by PMAM.Person_Master_Id
 
 
union
 
 
select PMAM.Person_Master_Id ,
sum(t. average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM
inner join fincore_db.Fincore.account_master AM on PMAM.Account_Master_Id = am.Account_Master_Id
inner join (select beneficiaryAcNo ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY beneficiaryAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsThree 
where  
beneficiaryAcNo is not null and beneficiaryAcNo !=''
and transactionType ='credit'
group by beneficiaryAcNo ,transactionDate)t 
on t.beneficiaryAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.Person_Master_Id




union 


select PMAM.Person_Master_Id ,
sum(t. average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM
inner join fincore_db.Fincore.account_master AM on PMAM.Account_Master_Id = am.Account_Master_Id
inner join (select beneficiaryAcNo ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY beneficiaryAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_GenReporting
where  
beneficiaryAcNo is not null and beneficiaryAcNo !=''
group by beneficiaryAcNo ,transactionDate)t 
on t.beneficiaryAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.Person_Master_Id


)w
group by w.person_master_id
)X

where FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id = X.Person_Master_Id'''


per_trx_update_debit = '''
update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set TRAN_AMNT_DEBIT = round(average,2)
from (
select w.Person_Master_Id as Person_Master_Id,sum(w.monthlyaverage)as average
from 
(select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM
inner join fincore_db.Fincore.account_master AM on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select accountNumber ,
avg(amount) as average,
RANK() OVER (PARTITION BY accountNumber ORDER BY  transactionDate desc ) Rank
--sum(amount)/Datediff(Month,transactionDate,getdate())  as monthlyaverage
from  FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_Tctwo 
where
accountNumber is not null and accountNumber !=''
and depositeWithdrawl ='Withdrawal'
group by accountNumber,transactionDate)t on t.accountNumber=Am.Account_Number 
group by PMAM.Person_Master_Id
 
union
 
select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM
inner join fincore_db.Fincore.account_master AM on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select accountNumber ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY accountNumber ORDER BY  transactionDate desc ) Rank
from  FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TcOne 
where 
accountNumber is not null and accountNumber !=''
and depositWithdrawal ='Withdrawal'
group by accountNumber,transactionDate)t on t.accountNumber=Am.Account_Number 
group by PMAM.Person_Master_Id

union

select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM
inner join fincore_db.Fincore.account_master AM on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select SENDERACNO ,
AVG(Amount) as average,
RANK() OVER (PARTITION BY SENDERACNO ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsOne 
where  
SENDERACNO is not null and SENDERACNO !=''
and transactionType ='DEBIT'
group by SENDERACNO ,transactionDate)t
on t.SENDERACNO=Am.Account_Number 
where Rank = 1
group by PMAM.Person_Master_Id
 
union
 
select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM
inner join fincore_db.Fincore.account_master AM on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select senderAcNo ,
AVG (amount) as average,
RANK() OVER (PARTITION BY senderAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsTwo 
where 
senderAcNo is not null and senderAcNo !=''
and transactionType ='DEBIT'
group by senderAcNo ,transactionDate)t
on t.senderAcNo=Am.Account_Number 
where Rank  = 1
group by PMAM.Person_Master_Id
 
union

select PMAM.Person_Master_Id ,
sum(t. average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM
inner join fincore_db.Fincore.account_master AM on PMAM.Account_Master_Id = am.Account_Master_Id
inner join (select senderAcNo ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY senderAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsThree 
where  
senderAcNo is not null and senderAcNo !=''
and transactionType ='DEBIT'
group by senderAcNo ,transactionDate)t 
on t.senderAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.Person_Master_Id

union 

select PMAM.Person_Master_Id ,
sum(t. average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM
inner join fincore_db.Fincore.account_master AM on PMAM.Account_Master_Id = am.Account_Master_Id
inner join (select senderAcNo ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY senderAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_GenReporting
where  
senderAcNo is not null and senderAcNo !=''
group by senderAcNo ,transactionDate)t 
on t.senderAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.Person_Master_Id


)w
group by w.person_master_id
)X
where 
FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id = X.Person_Master_Id'''

peer_grouping_columns = '''update fincore_db.fincore_analytics.RISK_PERSON_SUMMARY_STG
set PEER_GROUP_AMNT  = Case 
when (CBDT_INCOME_RANGE IS not null and CBDT_INCOME_RANGE!=0) then CBDT_INCOME_RANGE
when (TRAN_AMNT_CREDIT Is not null and TRAN_AMNT_CREDIT!=0) then TRAN_AMNT_CREDIT
when (TRAN_AMNT_DEBIT IS not null and TRAN_AMNT_DEBIT!=0) then TRAN_AMNT_DEBIT
else 0
end 
'''


In [8]:
# ------Mobile Count 
MOBILE_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set MOBILE_COUNT= k.no_of_contacts from 
(
SELECT  distinct  PMCM.Person_Master_Id,  
      Count(Distinct PMCM.Contact_Master_Id) as no_of_contacts    
      FROM [FINCORE_DB].[Fincore].Person_Master_Contact_Master PMCM  with(nolock)
      INNER JOIN [FINCORE_DB].[Fincore].Contact_Master CM      with(nolock)
      ON PMCM.Contact_Master_Id = CM.Contact_Master_Id 
      WHERE PMCM.Is_Active_Flag = 1 AND CM.Contact_Type = 'M'  GROUP BY PMCM.Person_Master_Id)k
      where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''


# ------TELEPHONE Count 
TELEPHONE_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set TELEPHONE_COUNT= k.no_of_contacts from 
(
SELECT  distinct  PMCM.Person_Master_Id,  
      Count(Distinct PMCM.Contact_Master_Id) as no_of_contacts    
      FROM [FINCORE_DB].[Fincore].Person_Master_Contact_Master PMCM  with(nolock)
      INNER JOIN [FINCORE_DB].[Fincore].Contact_Master CM      with(nolock)
      ON PMCM.Contact_Master_Id = CM.Contact_Master_Id 
      WHERE PMCM.Is_Active_Flag = 1 AND CM.Contact_Type = 'T'  GROUP BY PMCM.Person_Master_Id)k
      where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''

# ------Email Count 
EMAIL_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set EMAIL_COUNT= k.no_of_emailid from 
(
SELECT B.PERSON_MASTER_ID,  
  COUNT(DISTINCT A.EMAIL_ID) as no_of_emailid       
  FROM  [FINCORE_DB].[Fincore].PERSON_CLUSTER A     with(nolock)
  INNER JOIN  [FINCORE_DB].[Fincore].PERSON_MASTER_CLUSTER_FINGATE_REPORT B   with(nolock)
  ON A.PERSON_CLUSTER_ID = B.PERSON_CLUSTER_ID       
  WHERE A.EMAIL_ID IS NOT NULL    and A.status_active_flag = 1 and    B.is_active_flag =1  
  GROUP BY B.Person_Master_Id)k
      where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''


# ------WATCHLIST COUNT 
WATCHLIST_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set WATCHLIST_COUNT= k.watchlist_count from 
(
select  pm.Person_Master_Id,     
               count(w.Agency_Name) as watchlist_count         
               from  FINCORE_DB.Fincore.Person_Master PM       with(nolock)   
               inner join [FINCORE_DB].[Fincore].[Watchlist] W      with(nolock)  
               on PM.PERSON_MASTER_ID = W.Entity_Master_Id            
               where w.Entity_Type = 'I'   and PM.Is_Active_Flag =  1   and W.Is_Active_Flag = 1 group by pm.Person_Master_Id )k
                 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''


# -----STR_REPORT_COUNT
STR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set STR_REPORT_COUNT = K.STR_COUNT from
(SELECT   PMCFR.Person_Master_Id,      
                       COUNT( 1 ) AS STR_COUNT      
                       FROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'STR'  GROUP BY PMCFR.person_Master_Id )K
                       where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''


# -----CTR_REPORT_COUNT
CTR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set CTR_REPORT_COUNT = K.CTR_COUNT from
(SELECT   PMCFR.Person_Master_Id,      
                       COUNT( 1 ) AS CTR_COUNT      
                       FROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR with (nolock) 
                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CTR'  GROUP BY PMCFR.person_Master_Id )K
                       where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''


# -----NTR_REPORT_COUNT
NTR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set NTR_REPORT_COUNT = K.NTR_COUNT from
(SELECT   PMCFR.Person_Master_Id,      
                       COUNT( 1 ) AS NTR_COUNT      
                       FROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'NTR'  GROUP BY PMCFR.person_Master_Id )K
                       where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''


# -----PTR_REPORT_COUNT
PTR_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set PTR_REPORT_COUNT = K.PTR_COUNT from
(SELECT   PMCFR.Person_Master_Id,      
                       COUNT( 1 ) AS PTR_COUNT      
                       FROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'PTR'  GROUP BY PMCFR.person_Master_Id )K
                       where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''


# ---------------CBWTR_REPORT_COUNT
CBWTR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set CBWTR_REPORT_COUNT = K.CBWTR_COUNT from
(SELECT   PMCFR.Person_Master_Id,      
                       COUNT( 1 ) AS CBWTR_COUNT      
                       FROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CBWTR'  GROUP BY PMCFR.person_Master_Id )K
                       where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''


# -----CCR_REPORT_COUNT
CCR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set CCR_REPORT_COUNT = K.CCR_COUNT from
(SELECT   PMCFR.Person_Master_Id,      
                       COUNT( 1 ) AS CCR_COUNT      
                       FROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CCR'  GROUP BY PMCFR.person_Master_Id )K
                       where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''


# ----str_amount_debit
str_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set str_amount_debit = k.str_amount_debit from


(select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as str_amount_debit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
(( PMCFR.Is_Active_Flag = 1 and  B.typeName = 'STR') and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''

#      ----str_amount_credit

str_amount_credit =  '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set str_amount_credit = k.str_amount_credit from

 (select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as str_amount_credit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with (nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'STR' )and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''

# ----ctr_amount_debit
ctr_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set ctr_amount_debit = k.ctr_amount_debit from


(select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as ctr_amount_debit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CTR') and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''

#  ----ctr_amount_credit

ctr_amount_credit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set ctr_amount_credit = k.ctr_amount_credit from

 (select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as ctr_amount_credit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CTR') and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''

# ----ntr_amount_debit
ntr_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set ntr_amount_debit = k.ntr_amount_debit from


(select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as ntr_amount_debit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'NTR' )and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''

#      ----ntr_amount_credit

ntr_amount_credit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set ntr_amount_credit = k.ntr_amount_credit from
 (select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as ntr_amount_credit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'NTR') and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''

# ----cbwtr_amount_debit
cbwtr_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set cbwtr_amount_debit = k.cbwtr_amount_debit from


(select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as cbwtr_amount_debit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  t.REPORT_TYPE = 'CBWTR') and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''

#      ----cbwtr_amount_credit

cbwtr_amount_credit = ''' update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set cbwtr_amount_credit = k.cbwtr_amount_credit from

 (select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as cbwtr_amount_credit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  t.REPORT_TYPE = 'CBWTR') and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''

# ----ccr_amount_debit
ccr_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set ccr_amount_debit = k.ccr_amount_debit from


(select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as ccr_amount_debit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  t.REPORT_TYPE = 'CCR') and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''

#      ----ccr_amount_credit
ccr_amount_credit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set ccr_amount_credit = k.ccr_amount_credit from
 (select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as ccr_amount_credit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  t.REPORT_TYPE = 'CCR') and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''

In [9]:
tran_amnt = '''UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG SET TRAN_AMNT_DEBIT = T.TRAN_DEBIT
FROM (
SELECT ENTITY_ID,ENTITY_TYPE,CR_DR_DEP_WITH,SUM(TRAN_AMNT) AS TRAN_DEBIT   
 FROM  FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG  RTSS
 INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_INCREMENTAL K
 ON RTSS.ENTITY_ID = K.PERSON_MASTER_ID AND ENTITY_TYPE = 'I' 
GROUP BY ENTITY_ID,ENTITY_TYPE,CR_DR_DEP_WITH
HAVING CR_DR_DEP_WITH = 'DEBIT'
)T
WHERE ENTITY_ID = PERSON_MASTER_ID


UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG SET TRAN_AMNT_CREDIT = T.TRAN_CREDIT
FROM (
SELECT ENTITY_ID,ENTITY_TYPE,COALESCE(CR_DR_DEP_WITH,'CREDIT') AS CR_DR_DEP_WITH ,SUM(TRAN_AMNT) AS TRAN_CREDIT   
 FROM  FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG RTSS  
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_INCREMENTAL K
 ON RTSS.ENTITY_ID = K.PERSON_MASTER_ID AND ENTITY_TYPE = 'I' 
GROUP BY ENTITY_ID,ENTITY_TYPE,COALESCE(CR_DR_DEP_WITH,'CREDIT')
HAVING COALESCE(CR_DR_DEP_WITH,'CREDIT') = 'CREDIT'
)T
WHERE ENTITY_ID = PERSON_MASTER_ID'''


In [10]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]


In [21]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [22]:
#Run these queries to populate risk_person_summary_stg table

cursor.execute('''DELETE FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_Incremental] ''')
conn.commit()
cursor.execute('''INSERT INTO [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_Incremental] 

SELECT DISTINCT PERSON_MASTER_ID
FROM FINCORE.PERSON_MASTER_CLUSTER_FINGATE_REPORT PMCFR WITH (NOLOCK)
INNER JOIN FINCORE.PERSON_CLUSTER PC  WITH (NOLOCK)
ON PMCFR.PERSON_CLUSTER_ID = PC.PERSON_CLUSTER_ID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_PERSONDETAIL PD
ON PC.FINGATE_PERSON_ID = PD.ID_ AND PD.REPORTID = PMCFR.REPORT_ID
where is_active_flag = 1
AND 
COALESCE (PMCFR.UPDATE_DATE,PMCFR.CREATE_DATE) > COALESCE (( SELECT MAX( SOURCE_MAX_DATE ) FROM FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN
WHERE JOB_STATUS='SUCCESS' AND JOB_NAME='RISK_PERSON' )
,CAST('1900-01-01 00:00:00.0000000' AS DATETIME2)) ''') 

conn.commit()



per_queries = [master_data_pop_per_q1,master_data_pop_per_q2,per_pincode_update,per_occ_update,master_data_pop_per_income,tran_amnt,peer_grouping_columns,MOBILE_COUNT,TELEPHONE_COUNT,EMAIL_COUNT,WATCHLIST_COUNT,STR_REPORT_COUNT,CTR_REPORT_COUNT,NTR_REPORT_COUNT,PTR_COUNT,CBWTR_REPORT_COUNT,CCR_REPORT_COUNT,str_amount_debit,str_amount_credit,ctr_amount_debit,ctr_amount_credit,ntr_amount_debit,ntr_amount_credit,cbwtr_amount_debit,cbwtr_amount_credit,ccr_amount_debit,ccr_amount_credit]
for query in per_queries:
    cursor.execute(query)
    print(namestr(query,globals())[0]," DONE!")
    conn.commit()
    
    


conn.close()

master_data_pop_per_q1  DONE!
master_data_pop_per_q2  DONE!
per_pincode_update  DONE!
per_occ_update  DONE!
master_data_pop_per_income  DONE!
per_trx_update_credit  DONE!
per_trx_update_debit  DONE!
peer_grouping_columns  DONE!
MOBILE_COUNT  DONE!
TELEPHONE_COUNT  DONE!
EMAIL_COUNT  DONE!
WATCHLIST_COUNT  DONE!
STR_REPORT_COUNT  DONE!
CTR_REPORT_COUNT  DONE!
NTR_REPORT_COUNT  DONE!
PTR_COUNT  DONE!
CBWTR_REPORT_COUNT  DONE!
CCR_REPORT_COUNT  DONE!
str_amount_debit  DONE!
str_amount_credit  DONE!
ctr_amount_debit  DONE!
ctr_amount_credit  DONE!
ntr_amount_debit  DONE!
ntr_amount_credit  DONE!
cbwtr_amount_debit  DONE!
cbwtr_amount_credit  DONE!
ccr_amount_debit  DONE!
ccr_amount_credit  DONE!


In [13]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [14]:
#Populate the risk_person_summary_stg_rules
for query in rules_df[(rules_df['IS_ACTIVE_FLAG'] == 1) & (rules_df['PERSON_TYPE'] == 'Individual')]['Query'].to_list():
    pprint(query)
    cursor.execute(query)
    print("Done")
    conn.commit()
    
conn.close()

('\r\n'
 '   MERGE INTO '
 'FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG_RULES  \r\n'
 '   WITH (HOLDLOCK) as target USING     \r\n'
 '   (   \r\n'
 '   SELECT   PMAM.Person_Master_Id,    \r\n'
 '   Count(Distinct PMAM.Account_Master_Id) as No_Of_Accounts  \r\n'
 '   FROM [FINCORE_DB].[Fincore].Person_Master_Account_Master PMAM  with '
 '(nolock)     \r\n'
 '   INNER JOIN [FINCORE_DB].[Fincore].Account_Master AM     with '
 '(nolock)      \r\n'
 '   ON PMAM.Account_Master_Id = AM.Account_Master_Id  inner join \r\n'
 '   FINCORE_DB.Fincore_Analytics.RISK_PERSON_INCREMENTAL k on '
 'PMAM.Person_Master_Id  = k.person_master_id\r\n'
 '   WHERE PMAM.Is_Active_Flag = 1     \r\n'
 "   and AM.Account_Status = 'Active'      \r\n"
 "   and AM.account_type in   ('Current Account','Savings Account',\r\n"
 "   'Loan Account','Cash Credit Account','Overdraft Account','Term Deposit "
 "Account','NRE Account','NRO Account') \r\n"
 '   GROUP BY PMAM.Person_Master_Id)        AS source   \r\n'
 '

Done
('                         \r\n'
 '       MERGE INTO '
 'FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG_RULES  \r\n'
 '\t   WITH (HOLDLOCK) as target USING (   \r\n'
 '\t   select  pmam.Person_Master_Id,count(am.Account_Master_Id) as '
 'No_of_wealth_account \r\n'
 '\t   from FINCORE_DB.Fincore.Account_Master am  with (nolock) \r\n'
 '\t   inner join FINCORE_DB.Fincore.Person_Master_Account_Master pmam with '
 '(nolock) on am.Account_Master_Id=pmam.Account_Master_Id   \r\n'
 '\t   inner join FINCORE_DB.Fincore_Analytics.RISK_PERSON_INCREMENTAL k with '
 '(nolock) on PMAM.Person_Master_Id  = k.person_master_id    \r\n'
 "\t   where am.Account_Type like 'P%' or am.Account_Type like 'W%' \r\n"
 '\t   group by pmam.Person_Master_Id) as source    \r\n'
 '\t   ON target.PERSON_MASTER_ID = source.PERSON_master_id  and '
 "target.rule_no='P002001021' \r\n"
 '\t   WHEN MATCHED THEN   \r\n'
 '\t   UPDATE SET target.RULE_VALUE=source.No_of_wealth_account WHEN NOT '
 'MATCHED BY TARGET 

Done
('\r\n'
 '          MERGE INTO '
 'FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG_RULES as target USING '
 '(       \r\n'
 '\t\t  select temp.Person_Master_Id ,      \r\n'
 '\t\t  temp.Income_Per_Year_Inr,   \r\n'
 '\t\t  case when temp.Income_Per_Year_Inr is not null then '
 '((sum(temp.property_value)-temp.Income_Per_Year_Inr)/temp.Income_Per_Year_Inr)*100 '
 'end as   \r\n'
 '\t\t  Total_property_Value from   \r\n'
 '\t\t  (select pm.Person_Master_Id     ,     \r\n'
 '\t\t  cbdtir.Income_Per_Year_Inr, \r\n'
 '\t\t  sum(fgptr.amount) as property_value from '
 'fincore_db.Fincore.Person_Master pm   \r\n'
 '\t\t  inner join fincore_bridge_db.fincore_bridge.fingate_ptr fgptr on '
 'pm.pan = fgptr.buyerPan     \r\n'
 '\t\t  inner join Fincore_db.Fincore.Person_Master_Cluster_Fingate_Report '
 'pmcfr on pm.Person_Master_Id=pmcfr.Person_Master_Id   \r\n'
 '\t\t  inner join fincore_db.Fincore.Person_Cluster pc on '
 'pc.Person_Cluster_Id=pmcfr.Person_Cluster_Id      \r\n'
 '\t\t 

Done
('\r\n'
 '                MERGE INTO '
 '[FINCORE_DB].[Fincore_Analytics].[RISK_person_SUMMARY_STG_RULES]       \r\n'
 '\t\t\t\tas target USING (         \r\n'
 '\t\t\t\tSELECT  DISTINCT   B.PERSON_MASTER_ID \r\n'
 '\t\t\t\t,cast(CAST(MAX(C.createDate) AS DATE)as nvarchar) AS AGEING         '
 ',\r\n'
 '\t\t\t\tcase when YEAR(GETDATE()) - YEAR(C.createDate)>12 then 1\r\n'
 '\t\t\t\telse\r\n'
 '\r\n'
 '\t\t\t\tAVG(CAST( COALESCE(rlsr.RISK_SCORE,0) AS bigint))  end AS '
 'SCORE     \r\n'
 '\t\t\t\tFROM [FINCORE_DB].[Fincore].PERSON_CLUSTER A  with (nolock)    \r\n'
 '\t\t\t\tINNER JOIN '
 '[FINCORE_DB].[Fincore].PERSON_MASTER_CLUSTER_FINGATE_REPORT B with '
 '(nolock)  \r\n'
 '\t\t\t\tON A.PERSON_CLUSTER_ID = B.PERSON_CLUSTER_ID   inner '
 'join         \r\n'
 '\t\t\t\tFINCORE_DB.Fincore_Analytics.RISK_PERSON_INCREMENTAL k on '
 'b.Person_Master_Id  = k.person_master_id     \r\n'
 '\t\t\t\tINNER JOIN [FINCORE_BRIDGE_DB].[Fincore_BRIDGE].FINGATE_BATCHUPLOAD '
 'C with (nolock) \r\n'


In [18]:
print("Master Data Pop completed")

Master Data Pop completed
